In [4]:
# set your directory first
woz_directory = "./data/"
model_name = "bert-base-uncased"

---


# 1. Constructing Model




### Imports

In [1]:
# please add all your imports here
import re
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, BertTokenizer

## 1.1 Analyze the Training Data and Slots

- The following Code Cell parse the  training targets and find how many slots are the in the training data answer.


In [6]:
def parse_target(line):
    match = re.findall(r"(\w+-\w+)=([\w+\s*&*]+)", line)
    slot_name = [m[0] for m in match]
    slot_value = [m[1] for m in match]
    return slot_name, slot_value

In [8]:
with open(woz_directory + "WOZ_train_ans.txt") as f:
    train_answers = [answer.strip() for answer in f.readlines()]

slot_mapping = defaultdict(set)

for line in train_answers:
    slot_name, slot_value = parse_target(line)
    for i, name in enumerate(slot_name):
        slot_mapping[name].add(slot_value[i])

print(slot_mapping.keys())
print("-" * 50)

# for key in slot_mapping:
#     print(key, "-----", slot_mapping[key])
#     print(len(slot_mapping[key]))

dict_keys(['hotel-area', 'hotel-internet', 'hotel-parking', 'hotel-name', 'restaurant-food', 'restaurant-pricerange', 'restaurant-area', 'restaurant-name', 'hotel-pricerange', 'hotel-stars', 'hotel-type'])
--------------------------------------------------


### Statistics:
- **Slots for Hotel:** 4
    - {'stars', 'internet', 'parking', 'name', 'type', 'pricerange', 'area'}
- **Slots for Restaurant:** 7
    - {'pricerange', 'name', 'area', 'food'}
- We have 4+7 = **11 slots in total**

### The Slots can be divided into two categories:
- **Closed Class**(Slots that have fixed Values):
    - These slot values can be predefined and can be used to train a classifier.
    - The following slots are closed class slots:
```python
dict_keys(['hotel-area', 'hotel-internet', 'hotel-parking', 'restaurant-pricerange', 'restaurant-area', 'hotel-pricerange', 'hotel-stars', 'hotel-type'])
```
- **Open Class**(Slots that have variable values):
    - These slot values can be extracted using Named Entity Recognition(NER) or any other method.
    - Slots like `name`,`food` in both hotel and restaurant are open class slots.
```python
['hotel-name', 'restaurant-food','restaurant-name']
```
### Missing Value
- Sometimes the utterance suggests meaning like: "Anything will be fine", then the slot_value will be `dontcare`.
- That does not equals to situation when "slot not present in the target" . We need to handle not-present slot.
    - handled by adding "not present" to all the slots in `CLSOED_SLOTS[slot_name]`

## 1.2 Model Definition

###  Neural Network Model(BERT)

#### Divided the Problem
- After anallyzing the nature of the training target carefully. We realize that the problem can be divided into three parts:

**Intent Classification**:
- Find_hotel vs Find_restaurant
    
**Closed Class Slots-->sentence level classification:**
- Closed Class Slots: The value for closed sometimes is not present in the utterance. It relies on the context.
- The value is fixed, and the number of unique values is limited. This means the slot value can be predicted by convert **each closed class** slot to a **multi class** problem.

**Open Class Slots-->NER-->token level labeling**
- Convert this problem to Named Entity Recognition(NER) problem using BIO tagging. Because BERT can predict on sequence to sequence level.
- Because the slot value here is not fixed,
- and the number of unique values is large.
- We need to extract the slot value from the utterance.

#### Model Architecture:

In [9]:
class HybridSlot(nn.Module):
    def __init__(self, model_name, closed_slots, num_BIO_tags):
        super(HybridSlot, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.hidden_size = self.bert.config.hidden_size

        self.dropout = nn.Dropout(0.1)

        # intent classification layer, 2 for 2 intents
        self.intent = nn.Linear(self.hidden_size, 2)

        # closed class slots layer
        self.closed_slots = nn.ModuleDict(
            {
                slot_name: nn.Linear(self.hidden_size, len(slot_value))
                for slot_name, slot_value in closed_slots.items()
            }
        )

        # open class slots layer
        self.sequence_labeling = nn.Linear(self.hidden_size, num_BIO_tags)

    def forward(
        self,
        input_ids,
        attention_mask,
        intent_label=None,
        closed_slot_labels=None,
        open_slot_labels=None,
    ):

        # Get BERT outputs(predictions)
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        tags_prediction = outputs[
            0
        ]  # for open slots, [batch_size, tags_length[32], hidden_size]
        classification = outputs[
            1
        ]  # for intent and closed slots[batch_size, hidden_size]

        # Apply dropout
        classification = self.dropout(classification)
        tags_prediction = self.dropout(tags_prediction)

        # ----Get logits(probabilities) for all three tasks----

        # 1. Intent Classification
        intent_logits = self.intent(classification)  # [batch_size, 2]

        # 2. Closed slots classification
        closed_slots_logits = {}
        for slot_name, layer in self.closed_slots.items():
            closed_slots_logits[slot_name] = layer(classification)

        # 3. Open slots sequence labeling
        sequence_logits = self.sequence_labeling(
            tags_prediction
        )  # [batch_size, seq_length, num_BIO_tags]
        ## ---------------------------------------------------

        # -----------If we're training (labels provided), compute loss----------
        if intent_label is not None:
            loss_fct = nn.CrossEntropyLoss()
            total_loss = 0

            # 1.Intent classification loss
            intent_loss = loss_fct(intent_logits, intent_label)
            total_loss += intent_loss

            # 2.Closed slots classification loss
            closed_slots_loss = 0
            for slot_name in closed_slot_labels:
                # Skip examples where slot is not present (-1)
                valid_indices = closed_slot_labels[slot_name] != -1
                if valid_indices.sum() > 0:
                    valid_logits = closed_slots_logits[slot_name][valid_indices]
                    valid_labels = closed_slot_labels[slot_name][valid_indices]
                    closed_slots_loss += loss_fct(valid_logits, valid_labels)
            total_loss += closed_slots_loss

            # 3.Open slots sequence labeling loss
            # Only compute loss on valid tokens (exclude padding)
            active_loss = attention_mask.view(-1) == 1
            active_logits = sequence_logits.view(-1, sequence_logits.shape[-1])[
                active_loss
            ]
            active_labels = open_slot_labels.view(-1)[active_loss]

            # Exclude padding tokens from loss computation
            valid_label_mask = active_labels != 0  # Assuming 0 is padding index
            if valid_label_mask.sum() > 0:
                sequence_loss = loss_fct(
                    active_logits[valid_label_mask], active_labels[valid_label_mask]
                )
                total_loss += sequence_loss

            return total_loss

        # If we're not training (no labels), return predictions
        return intent_logits, closed_slots_logits, sequence_logits

# 2. Prepare Training data:

**Convert training data to format for model**

## 2.1 Load training data first

In [10]:
with open(woz_directory + "WOZ_train_utt.txt") as f:
    train_utterances = [answer.strip() for answer in f.readlines()]

with open(woz_directory + "WOZ_train_ans.txt") as f:
    train_answers = [answer.strip() for answer in f.readlines()]

print("Length of train_utterances:", len(train_utterances))
print("Length of train_answers:", len(train_answers))

# initialze a tokenizer, we will use this to tokenize the utterance when extracting slots
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Length of train_utterances: 3760
Length of train_answers: 3760


#### Examining some meta data:
```python
# all possible values for closed_slots
# I add a "notMentioned"! in case slot is missing in the data
CLOSED_SLOTS = {
        hotel-area : {'centre', 'south', 'notMentioned', 'dontcare', 'east', 'west', 'north'} ,
        hotel-internet : {'no', 'notMentioned', 'yes', 'dontcare'} ,
        hotel-parking : {'no', 'notMentioned', 'yes', 'dontcare'} ,
        restaurant-pricerange : {'expensive', 'notMentioned', 'dontcare', 'cheap', 'moderate'} ,
        restaurant-area : {'centre', 'south', 'notMentioned', 'dontcare', 'east', 'west', 'north'} ,
        hotel-pricerange : {'expensive', 'notMentioned', 'dontcare', 'cheap', 'moderate'} ,
        hotel-stars : {'3', '2', 'notMentioned', 'dontcare', '4', '5', '1', '0'} ,
        hotel-type : {'notMentioned', 'guesthouse', 'dontcare'} ,
    }
```

In [11]:
OPEN_SLOT_NAMES = ["hotel-name", "restaurant-food", "restaurant-name"]
CLOSED_SLOT_NAMES = [
    "hotel-area",
    "hotel-internet",
    "hotel-parking",
    "restaurant-pricerange",
    "restaurant-area",
    "hotel-pricerange",
    "hotel-stars",
    "hotel-type",
]


OPEN_SLOTS = defaultdict(set)
CLOSED_SLOTS = slot_mapping.copy()  # WE KEEP THESE TO PASS TO BERT MODEL

for slot_name in OPEN_SLOT_NAMES:
    OPEN_SLOTS[slot_name] = slot_mapping[slot_name]
    # remove the open slots from closed slots
    CLOSED_SLOTS.pop(slot_name)


# to handle missing slots
for slot in CLOSED_SLOTS:
    CLOSED_SLOTS[slot].add("notMentioned")
    print(slot, ":", CLOSED_SLOTS[slot], ",")

hotel-area : {'dontcare', 'notMentioned', 'south', 'east', 'north', 'west', 'centre'} ,
hotel-internet : {'yes', 'dontcare', 'no', 'notMentioned'} ,
hotel-parking : {'yes', 'dontcare', 'no', 'notMentioned'} ,
restaurant-pricerange : {'dontcare', 'notMentioned', 'cheap', 'moderate', 'expensive'} ,
restaurant-area : {'dontcare', 'notMentioned', 'south', 'east', 'north', 'west', 'centre'} ,
hotel-pricerange : {'dontcare', 'notMentioned', 'cheap', 'moderate', 'expensive'} ,
hotel-stars : {'dontcare', '4', '3', 'notMentioned', '5', '2', '1', '0'} ,
hotel-type : {'dontcare', 'notMentioned', 'guesthouse'} ,


## 2.2 Extract target from Training data

### Extract Intent from single example

**Intent mapping**
```python
intent = {
        "find_hotel": 0,
        "find_restaurant": 1
        }
```

In [12]:
def extract_intent(answer):
    """convert intent for a single example"""
    intent = answer.split("|")[0]
    if intent == "find_hotel":
        return 0
    else:
        return 1

### Extract slots from single example

**Closed Slots:**
```python
closed_slots = {'hotel-area': 'centre', 'hotel-internet': 'yes', 'hotel-parking': 'yes'}
```

**Open Slots: convert slot to BIO-tag**

```python
utterance = "Hi there! Can you give me some info on Cityroomz?"
answer = "find_hotel|hotel-name=cityroomz" # ---> conver to BIO tag by

#1. split utterance
tokens = ['Hi', 'there!', 'Can', 'you', 'give', 'me', 'some', 'info', 'on', 'Cityroomz?']

# 2.tag them
slot_tag = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-hotel-name']

# after Bert Tokenizer, words will be splitted into differnt subword/morphemes, So we will need to align the tags

```

**the output of this funtion look like:**
```python
closed_slots = {'hotel-area': 'centre', 'hotel-internet': 'yes', 'hotel-parking': 'yes'}
BIO_tag = ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-hotel-name']
```

In [13]:
# tokenize sentence


def extract_slot(answer, utterance, tokenizer, CLOSED_SLOT_NAMES, OPEN_SLOT_NAMES):
    """convert closed_slot, open_slot for a single example"""

    closed_slots = {}
    tokens = tokenizer.tokenize(utterance)

    # initialize BIO tag to contain only 'O'
    BIO_tag = ["O"] * len(tokens)

    slot_names, slot_values = parse_target(answer)

    for slot_name, slot_value in zip(slot_names, slot_values):
        if slot_name in CLOSED_SLOT_NAMES:
            closed_slots[slot_name] = slot_value

        elif slot_name in OPEN_SLOT_NAMES:
            # giving tags to the tokenized utterance
            slot_subwords = tokenizer.tokenize(slot_value)

            # find out which part of the tokens match the slot_value_tokenized
            # give them tags(slot_name)
            for i in range(len(tokens) - len(slot_subwords) + 1):
                if tokens[i : i + len(slot_subwords)] == slot_subwords:
                    BIO_tag[i] = "B-" + slot_name
                    for j in range(1, len(slot_subwords)):
                        BIO_tag[i + j] = "I-" + slot_name
    # print(tokens)
    # print(BIO_tag)
    return closed_slots, BIO_tag


# i = 2
# extract_slot(train_answers[i], train_utterances[i],tokenizer,CLOSED_SLOT_NAMES, OPEN_SLOT_NAMES)

### Pack examples from`train_X and train_y`

**Sample parsed training answer**

```python
example = {
        "intent" : 0,
        "closed_slots": {
                        "hotel-area" : 'north',
                        "hotel-internet" : 'yes'
                        },
        "open_slots" : ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-hotel-name']
    }
```

In [14]:
def pack_examples(
    train_answers, train_utterances, tokenizer, CLOSED_SLOT_NAMES, OPEN_SLOT_NAMES
):
    examples = []
    for answer, utterance in zip(train_answers, train_utterances):
        example = {}

        example["utterance"] = utterance

        intent = extract_intent(answer)
        example["intent"] = intent

        closed_slots, BIO_tags = extract_slot(
            answer, utterance, tokenizer, CLOSED_SLOT_NAMES, OPEN_SLOT_NAMES
        )

        example["closed_slots"] = closed_slots
        example["open_slots"] = BIO_tags
        examples.append(example)

    return examples

### Convert the values to id using above Mapping

In [15]:
value2id_MAPPING = {
    slot_name: {v: i for i, v in enumerate(slot_mapping[slot_name])}
    for slot_name in CLOSED_SLOT_NAMES
}

id2value_MAPPING = {
    slot_name: {i: v for i, v in enumerate(slot_mapping[slot_name])}
    for slot_name in CLOSED_SLOT_NAMES
}


tag2id = {
    "[PAD]": 0,  # padding
    "O": 1,
    "B-hotel-name": 2,
    "I-hotel-name": 3,
    "B-restaurant-food": 4,
    "I-restaurant-food": 5,
    "B-restaurant-name": 6,
    "I-restaurant-name": 7,
}
id2tag = {v: k for k, v in tag2id.items()}

# print(value2id_MAPPING)
# print(id2value_MAPPING)

In [16]:
def convert_to_ids(examples, CLOSED_SLOT_NAMES, value2id_MAPPING):
    """convert the values"""
    converted_examples = []

    for example in examples:
        new_example = {}
        new_example["intent"] = example["intent"]
        new_example["utterance"] = example["utterance"]
        new_example["closed_slots"] = example[
            "closed_slots"
        ].copy()  # Create a deep copy to avoid modifying original
        # Create a deep copy to avoid modifying original

        # convert closed slots to ids
        for slot_name in CLOSED_SLOT_NAMES:
            if slot_name in example["closed_slots"]:
                # slot_name is like hotel-area, hotel-interne
                slot_value = example["closed_slots"][slot_name]
                # like yes, no, dontcare
                
                try:
                    new_example["closed_slots"][slot_name] = value2id_MAPPING[slot_name][slot_value]
                except KeyError:
                    new_example["closed_slots"][slot_name] = value2id_MAPPING[slot_name]["notMentioned"]

            else:
                new_example["closed_slots"][slot_name] = value2id_MAPPING[slot_name][
                    "notMentioned"
                ]  # 10 for slot not present

        open_ids = [tag2id[tag] for tag in example["open_slots"]]
        new_example["open_slots"] = open_ids

        converted_examples.append(new_example)

    return converted_examples

In [14]:
# example_ids = convert_to_ids(train_examples, CLOSED_SLOTS, value2id_MAPPING)
# example_ids[100]['closed_slots']

## Convert to batch Tensor

```python
batch = {
intent_label = torch.tensor([0, 1, 0, ..., 1])  # shape(batch_size,1)
,
closed_slots = {
    "hotel-area": torch.tensor([0, 2, 1, ..., 3]),      # shape [batch_size, 1]
    "hotel-internet": torch.tensor([1, 2, 0, ..., 2]),  # shape [batch_size, 1]
...
},
open_slot_labels = torch.tensor([  # shape[batch_size, max_length]
    [0, 0, 1, 2, 0, ..., 0],  # tags of first utterance
    [0, 3, 4, 0, ..., 0],     # tags of second utteranc
    ... # 16 utterance in example
])

}
```

In [15]:
# # Check max length in your dataset
# max_len = 0
# total_len = 0
# for example in train_examples:
#     tokens = tokenizer.tokenize(example["utterance"])
#     max_len = max(max_len, len(tokens))
#     total_len += len(tokens)
# print("total length of tokens", total_len)
# print(f"Average sequence length in dataset: {total_len / len(train_examples)}")
# print(f"Maximum sequence length in dataset: {max_len}")

In [21]:
BATCH_SIZE = 32
MAX_LEN = 128  # because the maximum length of the sequence is below 128


def collate_fn(batched_examples, MAX_LEN):
    """convert the examples to batch tensor"""
    # Tokenize all utterances in the batch
    utterances = [example["utterance"] for example in batched_examples]
    encoded = tokenizer(
        utterances,
        padding=True,
        truncation=False,
        max_length=MAX_LEN,
        return_tensors="pt",
    )

    # Get the actual sequence length after tokenization
    seq_length = encoded["input_ids"].size(1)

    batch = {
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
        "intent": torch.tensor([example["intent"] for example in batched_examples]),
    }

    # convert closed slots to batches
    batch["closed_slots"] = {}
    for slot_name in CLOSED_SLOT_NAMES:
        batch["closed_slots"][slot_name] = torch.tensor(
            [example["closed_slots"][slot_name] for example in batched_examples]
        )

    # covert open slots and pad to the SAME length as input_ids
    padded_open_slots = torch.zeros(len(batched_examples), seq_length, dtype=torch.long)
    for i, example in enumerate(batched_examples):
        length = min(len(example["open_slots"]), seq_length)
        padded_open_slots[i, :length] = torch.tensor(example["open_slots"][:length])

    batch["open_slots"] = padded_open_slots

    return batch

In [17]:
# collate_fn(example_ids[:BATCH_SIZE], MAX_LEN)

## Load to Dataset

In [22]:
class SlotFillingDataset(Dataset):
    def __init__(self, examples, tokenizer, max_len=32):
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]


def create_dataloader(utt_path, answers_path):
    with open(utt_path) as f:
        utterances = [line.strip() for line in f.readlines()]
    with open(answers_path) as f:
        answers = [line.strip() for line in f.readlines()]

    # 1.Create examples
    examples = pack_examples(
        answers, utterances, tokenizer, CLOSED_SLOT_NAMES, OPEN_SLOT_NAMES
    )

    # 2.Convert to ids
    examples_ids = convert_to_ids(examples, CLOSED_SLOTS, value2id_MAPPING)

    # 3.Create datasets
    dataset = SlotFillingDataset(examples_ids, tokenizer, MAX_LEN)

    # 4. Create data loaders
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        collate_fn=lambda x: collate_fn(x, MAX_LEN),
    )
    return loader

In [23]:
train_loader = create_dataloader(woz_directory + "WOZ_train_utt.txt", woz_directory + "WOZ_train_ans.txt")
dev_loader = create_dataloader(woz_directory + "WOZ_dev_utt.txt", woz_directory + "WOZ_dev_ans.txt")
test_loader = create_dataloader(woz_directory + "WOZ_test_utt.txt", woz_directory + "WOZ_test_ans.txt")

In [24]:
# Get a single batch
for batch in train_loader:
    print("Batch Keys:", batch.keys())  # Check what keys are in the batch
    print("Input IDs:", batch["input_ids"].shape)  # Inspect tensor shape
    print("First 5 Input IDs:\n", batch["input_ids"][:5])  # Inspect first few examples
    print("First 5 Intent Labels:\n", batch["intent"][:5])
    print("First 5 Open Slot Labels:\n", batch["open_slots"][:5])
    break  # Stop after printing one batch


Batch Keys: dict_keys(['input_ids', 'attention_mask', 'intent', 'closed_slots', 'open_slots'])
Input IDs: torch.Size([32, 34])
First 5 Input IDs:
 tensor([[  101,  9535,  2368,  6415,  1010,  1045,  2572,  6595, 11585,  1999,
          4729,  1998,  2342,  1037,  2173,  2000,  3637,  1012,  1045,  2342,
          2489,  5581,  1998,  4274,  1012,   102,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101,  7632,  2045,   999,  2064,  2017,  2507,  2033,  2070, 18558,
          2006,  2103,  9954,  2480,  1029,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101,  1045,  2572,  2559,  2005,  1037,  3309,  2315,  2632, 23147,
         10024,  2100,  7410,  4113,  2160,  1012,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [  101,  1045,  2572

c:\Users\jonat\miniforge3\Lib\site-packages\transformers\tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


---
# 3. Training the Model

## 3.1 baseline model

In [25]:
num_BIO_tags = len(tag2id)  # pass this to BERT when initializing the model

model = HybridSlot(model_name, CLOSED_SLOTS, num_BIO_tags)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# for epoch in range(3):
#     model.train()
#     for batch in train_loader:
#         optimizer.zero_grad()
#         loss = model(
#             input_ids=batch["input_ids"],
#             attention_mask=batch["attention_mask"],
#             intent_label=batch["intent"],
#             closed_slot_labels=batch["closed_slots"],
#             open_slot_labels=batch["open_slots"],
#         )
#         loss.backward()
#         optimizer.step()

## 3.2 Evaluate on dev set

In [27]:
def evaluate_model(model, eval_loader, device="cuda"):
    model.eval()

    intent_preds, intent_true = [], []
    closed_slot_preds, closed_slot_true = {}, {}
    open_slot_preds, open_slot_true = [], []

    # Initialize prediction lists for each slot
    for slot_name in model.closed_slots.keys():
        closed_slot_preds[slot_name] = []
        closed_slot_true[slot_name] = []

    with torch.no_grad():
        for batch in eval_loader:
            # Move batch to device if needed
            # I didn't! because I'm using CPU
            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]

            # Get model predictions
            intent_logits, closed_slot_logits, sequence_logits = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
            )

            # Process intent predictions
            intent_pred_values = intent_logits.argmax(dim=-1).cpu().numpy()
            intent_preds.extend(intent_pred_values)
            intent_true.extend(batch["intent"].numpy())

            # Process closed slot predictions
            for slot_name, slot_preds in closed_slot_logits.items():
                pred_values = slot_preds.argmax(dim=-1).cpu().numpy()
                closed_slot_preds[slot_name].extend(pred_values)
                closed_slot_true[slot_name].extend(
                    batch["closed_slots"][slot_name].numpy()
                )

            # Process open slot predictions (BIO tagging)
            sequence_pred_values = sequence_logits.argmax(dim=-1).cpu().numpy()
            attention_mask_np = batch["attention_mask"].cpu().numpy()

            for i in range(len(sequence_pred_values)):
                true_labels = batch["open_slots"][i].cpu().numpy()
                pred_labels = sequence_pred_values[i]

                # Mask padding tokens (only keep real words)
                actual_seq_len = attention_mask_np[i].sum()
                open_slot_true.extend(true_labels[:actual_seq_len])
                open_slot_preds.extend(pred_labels[:actual_seq_len])

    intent_accuracy = accuracy_score(intent_true, intent_preds)
    open_slot_metrics = f1_score(open_slot_true, open_slot_preds, average="macro")

    # print(f"Intent Accuracy: {intent_accuracy:.4f}")
    # print(f"Open Slot F1 Score: {open_slot_metrics:.4f}")

    closed_slot_metrics = {}
    # Calculate metrics for each closed slot
    for slot_name in model.closed_slots.keys():
        slot_preds = closed_slot_preds[slot_name]
        slot_labels = closed_slot_true[slot_name]

        closed_slot_metrics[slot_name] = f1_score(
            slot_labels, slot_preds, average="macro"
        )

    return intent_accuracy, closed_slot_metrics, open_slot_metrics


eval_results = evaluate_model(model, dev_loader)
print(f"Intent Accuracy: {eval_results[0]:.4f}")
print(f"Open Slot F1 Score: {eval_results[2]:.4f}\n")
for slot_name, f1 in eval_results[1].items():
    print(f"{slot_name} F1 Score: {f1:.4f}")

c:\Users\jonat\miniforge3\Lib\site-packages\transformers\tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Intent Accuracy: 0.4746
Open Slot F1 Score: 0.0219

hotel-area F1 Score: 0.0027
hotel-internet F1 Score: 0.0000
hotel-parking F1 Score: 0.0411
restaurant-pricerange F1 Score: 0.0220
restaurant-area F1 Score: 0.0061
hotel-pricerange F1 Score: 0.0093
hotel-stars F1 Score: 0.1120
hotel-type F1 Score: 0.0515


In [28]:
EPOCHS = 5
# Move batch to device if needed, I didn't! because I don't have one!

for epoch in range(EPOCHS):

    model.train()
    train_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        loss = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            intent_label=batch["intent"],
            closed_slot_labels=batch["closed_slots"],
            open_slot_labels=batch["open_slots"],
        )
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    # Evaluation phase
    eval_results = evaluate_model(model, dev_loader)

    # Print metrics
    print('-'*50)
    print(f"Epoch {epoch + 1}")
    print(f"Train Loss: {train_loss / len(train_loader):.4f}\n")
    print(f"Intent Accuracy: {eval_results[0]:.4f}")
    print(f"Open Slot F1 Score: {eval_results[2]:.4f}\n")
    for slot_name, f1 in eval_results[1].items():
        print(f"{slot_name} F1 Score: {f1:.4f}")

c:\Users\jonat\miniforge3\Lib\site-packages\transformers\tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


--------------------------------------------------
Epoch 1
Train Loss: 6.9138

Intent Accuracy: 0.9855
Open Slot F1 Score: 0.1137

hotel-area F1 Score: 0.1362
hotel-internet F1 Score: 0.4732
hotel-parking F1 Score: 0.2363
restaurant-pricerange F1 Score: 0.2206
restaurant-area F1 Score: 0.1448
hotel-pricerange F1 Score: 0.2373
hotel-stars F1 Score: 0.1914
hotel-type F1 Score: 0.4772


c:\Users\jonat\miniforge3\Lib\site-packages\transformers\tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


--------------------------------------------------
Epoch 2
Train Loss: 3.2681

Intent Accuracy: 0.9952
Open Slot F1 Score: 0.2978

hotel-area F1 Score: 0.1362
hotel-internet F1 Score: 0.9867
hotel-parking F1 Score: 0.4691
restaurant-pricerange F1 Score: 0.8002
restaurant-area F1 Score: 0.2550
hotel-pricerange F1 Score: 0.2373
hotel-stars F1 Score: 0.2446
hotel-type F1 Score: 0.9624


c:\Users\jonat\miniforge3\Lib\site-packages\transformers\tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


--------------------------------------------------
Epoch 3
Train Loss: 2.2676

Intent Accuracy: 0.9976
Open Slot F1 Score: 0.5629

hotel-area F1 Score: 0.1843
hotel-internet F1 Score: 0.9867
hotel-parking F1 Score: 0.4730
restaurant-pricerange F1 Score: 0.9580
restaurant-area F1 Score: 0.4103
hotel-pricerange F1 Score: 0.3045
hotel-stars F1 Score: 0.3512
hotel-type F1 Score: 0.9769


c:\Users\jonat\miniforge3\Lib\site-packages\transformers\tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


--------------------------------------------------
Epoch 4
Train Loss: 1.6595

Intent Accuracy: 0.9976
Open Slot F1 Score: 0.7609

hotel-area F1 Score: 0.2001
hotel-internet F1 Score: 0.9867
hotel-parking F1 Score: 0.4705
restaurant-pricerange F1 Score: 0.9778
restaurant-area F1 Score: 0.4206
hotel-pricerange F1 Score: 0.7787
hotel-stars F1 Score: 0.3434
hotel-type F1 Score: 0.9696


c:\Users\jonat\miniforge3\Lib\site-packages\transformers\tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


--------------------------------------------------
Epoch 5
Train Loss: 1.3072

Intent Accuracy: 0.9976
Open Slot F1 Score: 0.8222

hotel-area F1 Score: 0.3527
hotel-internet F1 Score: 0.9867
hotel-parking F1 Score: 0.4705
restaurant-pricerange F1 Score: 0.9752
restaurant-area F1 Score: 0.6080
hotel-pricerange F1 Score: 0.9429
hotel-stars F1 Score: 0.3405
hotel-type F1 Score: 0.9624


---
# 4. Make predictions

In [29]:
def get_predictions(model, data_loader, device="cpu"):
    model.eval()
    intent_preds = []
    closed_slot_preds = {}
    open_slot_preds = []

    # initiate closed slots dict
    for slot_name in model.closed_slots.keys():
        closed_slot_preds[slot_name] = []

    with torch.no_grad():
        for batch in data_loader:
            batch_x = {k: v for k, v in batch.items()}
            
            intent_logits, closed_slot_logits, sequence_logits = model(
                input_ids=batch_x["input_ids"],
                attention_mask=batch_x["attention_mask"],
            )

            intent_pred_values = intent_logits.argmax(dim=-1).cpu().numpy()
            intent_preds.extend(intent_pred_values)

            for slot_name, slot_logits in closed_slot_logits.items():
                pred_values = slot_logits.argmax(dim=-1).cpu().numpy()
                closed_slot_preds[slot_name].extend(pred_values)

            sequence_pred_values = sequence_logits.argmax(dim=-1).cpu().numpy()
            attention_mask_np = batch["attention_mask"].cpu().numpy()

            for i in range(len(sequence_pred_values)):
                pred_labels = sequence_pred_values[i]
                actual_seq_len = attention_mask_np[i].sum()
                open_slot_preds.append(pred_labels[:actual_seq_len])

    return intent_preds, closed_slot_preds, open_slot_preds


In [30]:
# run on GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
# print("Device:", device)

train_preds_intent, train_preds_closed, train_preds_open = get_predictions(model, train_loader)
dev_preds_intent, dev_preds_closed, dev_preds_open = get_predictions(model, dev_loader)

test_preds_intent, test_preds_closed, test_preds_open = get_predictions(model, test_loader)

c:\Users\jonat\miniforge3\Lib\site-packages\transformers\tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


---
# 5. Convert the model output to the final output format

In [31]:
def convert_intent(intent_preds):
    """take in a list of intent predctions"""
    return ["find_restaurant" if intent else "find_hotel" for intent in intent_preds]

def convert_closed_slots(closed_slot_preds, id2value):
    predictions = []
    """take in a dictionary of closed slot predictions"""
    # iterate over all examples
    for i in range(len(closed_slot_preds["hotel-area"])):
        results = {}
        for slot_name in closed_slot_preds:
            pred_id = closed_slot_preds[slot_name][i]
            pred_value = id2value[slot_name][pred_id]
            if pred_value != "notMentioned":
                results[slot_name] = pred_value
        predictions.append(results)
    return predictions

In [32]:
def extract_predicted_slots(open_slot_preds, input_ids, tokenizer):
    """
    Args:
        open_slot_preds: predictions, usually loader[2]
    Returns:
        extracted_texts: texted corresponding to predictions
        extracted_labels: id of predicted tag, like 3(id): hotel-name,
    """
    extracted_texts = []
    extracted_labels = []

    for idx, (preds, ids) in enumerate(zip(open_slot_preds, input_ids)):
        tokens = tokenizer.convert_ids_to_tokens(ids)
        slot_text = []
        slot_labels = []
        
        for token, pred in zip(tokens, preds):
            if pred > 1:
                slot_text.append(token)
                if pred % 2 == 0:
                    slot_labels.append(pred)
        
        if slot_text:
            extracted_texts.append(tokenizer.convert_tokens_to_string(slot_text))
            extracted_labels.append(slot_labels)

    return extracted_texts, extracted_labels

In [104]:
def convert_open_slots(open_slot_preds, slot_predictions, dataloader, tokenizer):

    for batch in dev_loader:
        input_ids = batch["input_ids"]
        # extract the open slot values
        extracted_texts, extracted_labels = extract_predicted_slots(
            open_slot_preds, input_ids, tokenizer
        )
        
        # adding open_slot predictions to the slot_predictions dict
        for i in range(len(extracted_texts)):
            tags = extracted_labels[i]
            for tag in tags:
                slot_name = id2tag[tag][2:]
                slot_predictions[i][slot_name] = extracted_texts[i]
    return slot_predictions

In [105]:
intent_predictions = convert_intent(dev_preds_intent)
slot_predictions = convert_closed_slots(dev_preds_closed, id2value_MAPPING)
slot_predictions = convert_open_slots(dev_preds_open, slot_predictions, dev_loader, tokenizer)

c:\Users\jonat\miniforge3\Lib\site-packages\transformers\tokenization_utils_base.py:2691: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [35]:
print(len(intent_predictions))
print(len(slot_predictions))
print(intent_predictions[:5])
slot_predictions[:5]


413
413
['find_restaurant', 'find_hotel', 'find_restaurant', 'find_hotel', 'find_hotel']


[{'restaurant-area': 'centre', 'restaurant-food': 'local'},
 {'restaurant-food': 'cheap'},
 {'restaurant-pricerange': 'expensive',
  'restaurant-area': 'centre',
  'restaurant-food': 'food'},
 {'hotel-parking': 'yes', 'restaurant-name': 'food restaurant in'},
 {'hotel-internet': 'yes',
  'hotel-pricerange': 'expensive',
  'restaurant-food': 'find'}]